In [307]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [308]:
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_selection import RFECV
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn import feature_selection
from sklearn import metrics
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

In [309]:
def load_afl_data():
    df_2017 = pd.read_csv("../data/afl_results_2017.csv")
    #print(df_2017.shape)
    df_2018 = pd.read_csv("../data/afl_results_2018.csv")
    #print(df_2018.shape)
    df_2019 = pd.read_csv("../data/afl_results_2019.csv")
    #print(df_2019.shape)
    df_2020 = pd.read_csv("../data/afl_results_2020.csv")
    #print(df_2020.shape)
    df_2021 = pd.read_csv("../data/afl_results_2021.csv")
    #print(df_2021.shape)
    df_2022 = pd.read_csv("../data/afl_results_2022.csv")
    #print(df_2022.shape)
    df_all = pd.concat([df_2017, df_2018, df_2019, df_2020, df_2021,df_2022], axis=0)
    df_all['Date'] = pd.to_datetime(df_all['match.date']).dt.strftime("%Y-%m-%d")
    df_players_2017 = pd.read_csv("../data/afl_players_stats_2017.csv")
    #print(df_players_2017.shape)
    df_players_2018 = pd.read_csv("../data/afl_players_stats_2018.csv")
    #print(df_players_2018.shape)
    df_players_2019 = pd.read_csv("../data/afl_players_stats_2019.csv")
    #print(df_players_2019.shape)
    df_players_2020 = pd.read_csv("../data/afl_players_stats_2020.csv")
    #print(df_players_2020.shape)
    df_players_2021 = pd.read_csv("../data/afl_players_stats_2021.csv")
    #print(df_players_2021.shape)
    df_players_2022 = pd.read_csv("../data/afl_players_stats_2022.csv")
    #print(df_players_2022.shape)
    df_players = pd.concat([df_players_2017, df_players_2018, df_players_2019,df_players_2020,df_players_2021,df_players_2022], axis=0)
    #print(df_players.shape)
    #df_players.columns
    df_fixture = pd.read_csv("../data/fixture_2022.csv")
    df_next_games_teams = df_fixture[(df_fixture['status'] != "CONCLUDED") & (df_fixture['round.roundNumber'] == 11)]
    df_next_games_teams = df_next_games_teams[['home.team.name','away.team.name','venue.name','compSeason.year','round.roundNumber']]
    df_next_games_teams = df_next_games_teams.rename(columns={'home.team.name': 'match.homeTeam.name', 'away.team.name': 'match.awayTeam.name','compSeason.year':'round.year'})
    df_next_games_teams['match.matchId'] = np.arange(len(df_next_games_teams))
    
    return df_all, df_players, df_fixture, df_next_games_teams

def get_aggregate_player_stats(df=None):

    agg_stats = (df.rename(columns={ # Rename columns to lowercase
                    'Home.team': 'match.homeTeam.name',
                    'Away.team': 'match.awayTeam.name',
                    })
                   .groupby(by=['Date', 'Season', 'match.homeTeam.name', 'match.awayTeam.name'], as_index=False) # Groupby to aggregate the stats for each game
                   .sum()
                   #.drop(columns=['DE', 'TOG', 'Match_id']) # Drop columns
                   .assign(date=lambda df: pd.to_datetime(df.Date, format="%Y-%m-%d")) # Create a datetime object
                   .sort_values(by='Date')
                   .reset_index(drop=True))
    return agg_stats

In [310]:
df_all, df_players, df_fixture, df_next_games_teams = load_afl_data()
agg_player = get_aggregate_player_stats(df_players)
afl_df = df_all.merge(agg_player, on=['Date', 'match.homeTeam.name', 'match.awayTeam.name'], how='left')
print(afl_df.shape)

(1072, 123)


In [311]:
# Add average goal diff for home and away team rolling 4 games

afl_df['HTGDIFF'] = afl_df['homeTeamScore.matchScore.goals'] - afl_df['awayTeamScore.matchScore.goals']
afl_df['ATGDIFF'] = afl_df['awayTeamScore.matchScore.goals'] - afl_df['homeTeamScore.matchScore.goals']

def from_dict_value_to_df(d):
    """
    input = dictionary 
    output = dataframe as part of all the values from the dictionary
    """
    df = pd.DataFrame()
    for v in d.values():
        df = pd.concat([df,v])
    return df

def avg_goal_diff(df, avg_h_a_diff, a_h_team, a_h_goal_letter):
    """
    input: 
        df = dataframe with all results
        avg_h_a_diff = name of the new column
        a_h_team = HomeTeam or AwayTeam
        a_h_goal_letter = 'H' for home or 'A' for away
    output: 
        avg_per_team = dictionary with with team as key and columns as values with new column H/ATGDIFF
    """
    df[avg_h_a_diff] = 0
    avg_per_team = {}
    all_teams = df[a_h_team].unique()
    for t in all_teams:
        df_team = df[df[a_h_team]==t].fillna(0)
        result = df_team['{}TGDIFF'.format(a_h_goal_letter)].rolling(4).mean()
        df_team[avg_h_a_diff] = result
        avg_per_team[t] = df_team
    return avg_per_team

d_AVGFTHG = avg_goal_diff(afl_df, 'AVGHTGDIFF', 'match.homeTeam.name', 'H')
df_AVGFTHG = from_dict_value_to_df(d_AVGFTHG)
df_AVGFTHG.sort_index(inplace=True)
d_AVGFTAG = avg_goal_diff(df_AVGFTHG, 'AVGATGDIFF', 'match.awayTeam.name', 'A')
afl_df = from_dict_value_to_df(d_AVGFTAG)
afl_df.sort_index(inplace=True)
afl_df['AVGATGDIFF'].fillna(0, inplace=True)




In [312]:
afl_df['goal_diff'] = afl_df['homeTeamScore.matchScore.goals'] - afl_df['awayTeamScore.matchScore.goals']

for index, row in df_all[df_all['match.status']=='CONCLUDED'].iterrows():
    if afl_df['goal_diff'][index] > 0:
        afl_df.at[index,'result'] = 1   # 1 is a win
    else:
        afl_df.at[index,'result'] = 0  # 0 is a loss 

In [313]:
def previous_data(df, h_or_a_team, column, letter, past_n):
    """
    input: 
        df = dataframe with all results
        a_h_team = HomeTeam or AwayTeam
        column = column selected to get previous data from
    output:
        team_with_past_dict = dictionary with team as a key and columns as values with new 
                              columns with past value
    """
    d = dict()
    team_with_past_dict = dict()
    all_teams = df[h_or_a_team].unique()
    for team in all_teams:
        n_games = len(df[df[h_or_a_team]==team])
        team_with_past_dict[team] = df[df[h_or_a_team]==team]
        for i in range(1, past_n):
            d[i] = team_with_past_dict[team].assign(
                result=team_with_past_dict[team].groupby(h_or_a_team)[column].shift(i)
            ).fillna({'{}_X'.format(column): 0})
            team_with_past_dict[team]['{}_{}_{}'.format(letter, column, i)] = d[i].result
    return team_with_past_dict

def previous_data_call(df, side, column, letter, iterations):
    d = previous_data(df, side, column, letter, iterations)
    df_result= from_dict_value_to_df(d)
    df_result.sort_index(inplace=True)
    return df_result

df_last_home_results = previous_data_call(afl_df, 'match.homeTeam.name', 'result', 'H', 3)
df_last_away_results = previous_data_call(df_last_home_results, 'match.awayTeam.name', 'result', 'A', 3)
df_last_last_HTGDIFF_results = previous_data_call(df_last_away_results, 'match.homeTeam.name', 'HTGDIFF', 'H', 3)
df_last_last_ATGDIFF_results = previous_data_call(df_last_last_HTGDIFF_results, 'match.awayTeam.name', 'ATGDIFF', 'A', 3)
df_last_AVGFTHG_results = previous_data_call(df_last_last_ATGDIFF_results, 'match.homeTeam.name', 'AVGHTGDIFF', 'H', 2)
df_last_AVGFTAG_results = previous_data_call(df_last_AVGFTHG_results, 'match.awayTeam.name', 'AVGATGDIFF', 'A', 2)
afl_df = df_last_AVGFTAG_results.copy()

In [314]:

all_cols = ['match.matchId','match.date', 'match.status', 'match.venue', 'match.homeTeam.name', 'match.awayTeam.name','venue.name', 'venue.state', 'round.name', 'round.year', 'round.roundNumber', 'status',
'homeTeamScore.rushedBehinds', 'homeTeamScore.minutesInFront',
       'homeTeamScore.matchScore.totalScore', 'homeTeamScore.matchScore.goals',
       'homeTeamScore.matchScore.behinds',
       'homeTeamScore.matchScore.superGoals', 'awayTeamScore.rushedBehinds',
       'awayTeamScore.minutesInFront', 'awayTeamScore.matchScore.totalScore',
       'awayTeamScore.matchScore.goals', 'awayTeamScore.matchScore.behinds',
       'awayTeamScore.matchScore.superGoals', 'weather.tempInCelsius',
       'homeTeamScoreChart.goals', 'homeTeamScoreChart.leftBehinds',
       'homeTeamScoreChart.rightBehinds', 'homeTeamScoreChart.leftPosters',
       'homeTeamScoreChart.rightPosters', 'homeTeamScoreChart.rushedBehinds',
       'homeTeamScoreChart.touchedBehinds', 'awayTeamScoreChart.goals',
       'awayTeamScoreChart.leftBehinds', 'awayTeamScoreChart.rightBehinds',
       'awayTeamScoreChart.leftPosters', 'awayTeamScoreChart.rightPosters',
       'awayTeamScoreChart.rushedBehinds', 'awayTeamScoreChart.touchedBehinds', 
       'HQ1G', 'HQ1B', 'HQ2G',
       'HQ2B', 'HQ3G', 'HQ3B', 'HQ4G', 'HQ4B', 'Home.score', 'AQ1G', 'AQ1B',
       'AQ2G', 'AQ2B', 'AQ3G', 'AQ3B', 'AQ4G', 'AQ4B', 'Away.score',
       'Kicks', 'Marks', 'Handballs', 'Goals', 'Behinds', 'Hit.Outs',
       'Tackles', 'Rebounds', 'Inside.50s', 'Clearances', 'Clangers',
       'Frees.For', 'Frees.Against', 'Brownlow.Votes', 'Contested.Possessions',
       'Uncontested.Possessions', 'Contested.Marks', 'Marks.Inside.50',
       'One.Percenters', 'Bounces', 'Goal.Assists', 'Time.on.Ground..',
       'Substitute', 'group_id', 'HTGDIFF', 'ATGDIFF', 'AVGHTGDIFF',
       'AVGATGDIFF', 'goal_diff', 'result', 'H_result_1', 'H_result_2',
       'A_result_1', 'A_result_2', 'H_HTGDIFF_1', 'H_HTGDIFF_2', 'A_ATGDIFF_1',
       'A_ATGDIFF_2', 'H_AVGHTGDIFF_1', 'A_AVGATGDIFF_1']

non_feature_cols = ['match.matchId','match.date', 'match.status', 'match.venue', 'match.homeTeam.name', 'match.awayTeam.name','venue.name', 'venue.state', 'round.name', 'round.year', 'round.roundNumber', 'status','Season']
feature_cols = [
       'homeTeamScore.rushedBehinds', 'homeTeamScore.minutesInFront',
       'homeTeamScore.matchScore.totalScore', 'homeTeamScore.matchScore.goals',
       'homeTeamScore.matchScore.behinds',
       'homeTeamScore.matchScore.superGoals', 'awayTeamScore.rushedBehinds',
       'awayTeamScore.minutesInFront', 'awayTeamScore.matchScore.totalScore',
       'awayTeamScore.matchScore.goals', 'awayTeamScore.matchScore.behinds',
       'awayTeamScore.matchScore.superGoals', 'weather.tempInCelsius',
       'homeTeamScoreChart.goals', 'homeTeamScoreChart.leftBehinds',
       'homeTeamScoreChart.rightBehinds', 'homeTeamScoreChart.leftPosters',
       'homeTeamScoreChart.rightPosters', 'homeTeamScoreChart.rushedBehinds',
       'homeTeamScoreChart.touchedBehinds', 'awayTeamScoreChart.goals',
       'awayTeamScoreChart.leftBehinds', 'awayTeamScoreChart.rightBehinds',
       'awayTeamScoreChart.leftPosters', 'awayTeamScoreChart.rightPosters',
       'awayTeamScoreChart.rushedBehinds', 'awayTeamScoreChart.touchedBehinds', 
       'HQ1G', 'HQ1B', 'HQ2G',
       'HQ2B', 'HQ3G', 'HQ3B', 'HQ4G', 'HQ4B', 'Home.score', 'AQ1G', 'AQ1B',
       'AQ2G', 'AQ2B', 'AQ3G', 'AQ3B', 'AQ4G', 'AQ4B', 'Away.score',
       'Kicks', 'Marks', 'Handballs', 'Goals', 'Behinds', 'Hit.Outs',
       'Tackles', 'Rebounds', 'Inside.50s', 'Clearances', 'Clangers',
       'Frees.For', 'Frees.Against', 'Brownlow.Votes', 'Contested.Possessions',
       'Uncontested.Possessions', 'Contested.Marks', 'Marks.Inside.50',
       'One.Percenters', 'Bounces', 'Goal.Assists', 'Time.on.Ground..',
       'Substitute', 'group_id', 'HTGDIFF', 'ATGDIFF', 'AVGHTGDIFF',
       'AVGATGDIFF', 'goal_diff', 'result', 'H_result_1', 'H_result_2',
       'A_result_1', 'A_result_2', 'H_HTGDIFF_1', 'H_HTGDIFF_2', 'A_ATGDIFF_1',
       'A_ATGDIFF_2', 'H_AVGHTGDIFF_1', 'A_AVGATGDIFF_1']

In [315]:
afl_df = afl_df[all_cols] 

afl_df = afl_df.rename(columns={col: 'f_' + col for col in afl_df if col not in non_feature_cols})


In [316]:
# Add new round data



In [317]:

def create_training_and_test_data(afl_df,df_next_games_teams):
    
    # Define a function which returns a DataFrame with the expontential moving average for each numeric stat
    def create_exp_weighted_avgs(df, span):
        # Create a copy of the df with only the game id and the team - we will add cols to this df
        ema_features = df[['match.matchId', 'match.homeTeam.name']].copy()

        feature_names = [col for col in df.columns if col.startswith('f_')] # Get a list of columns we will iterate over

        for feature_name in feature_names:
            feature_ema = (df.groupby('match.homeTeam.name')[feature_name]
                             .transform(lambda row: (row.ewm(span=span)
                                                        .mean()
                                                        .shift(1))))
            ema_features[feature_name] = feature_ema

        return ema_features
    
    
    afl_df['train_data'] = 1
    df_next_games_teams['train_data'] = 0
    
    afl_data = afl_df.append(df_next_games_teams).reset_index(drop=True)
    
    features_rolling_averages = create_exp_weighted_avgs(afl_data, span=10)
    
    features = afl_data[['match.date', 'match.matchId', 'match.homeTeam.name', 'match.awayTeam.name', 'venue.name','round.year','train_data']].copy()
    features = pd.merge(features, features_rolling_averages, on=['match.matchId', 'match.homeTeam.name'])
    
    form_btwn_teams = afl_df[['match.matchId', 'match.homeTeam.name', 'match.awayTeam.name', 'f_goal_diff']].copy()


#    form_btwn_teams_inv = pd.DataFrame()

#    for index, row in form_btwn_teams.iterrows():
#        home = row['match.homeTeam.name']
#        away = row['match.awayTeam.name']
#        matchid = row['match.matchId']
#        margin = row['f_goal_diff']

#        form_btwn_teams_inv = form_btwn_teams_inv.append({'match.matchId': matchid, 'match.homeTeam.name': away, 'match.awayTeam.name': home, 'f_goal_diff': -1*margin}, ignore_index=True)

#    form_btwn_teams['f_form_margin_btwn_teams'] = (form_btwn_teams.groupby(['match.homeTeam.name', 'match.awayTeam.name'])['f_goal_diff']
#                                                              .transform(lambda row: row.rolling(5).mean().shift())
#                                                              .fillna(0))

#    form_btwn_teams['f_form_past_5_btwn_teams'] = \
#    (form_btwn_teams.assign(win=lambda df: df.apply(lambda row: 1 if row.f_goal_diff > 0 else 0, axis='columns'))
#                  .groupby(['match.homeTeam.name', 'match.awayTeam.name'])['win']
#                  .transform(lambda row: row.rolling(5).mean().shift() * 5)
#                  .fillna(0))


    #print(features.shape)
    # Merge to our features df
    #features = pd.merge(features, form_btwn_teams_1.drop(columns=['f_goal_diff']), on=['match.matchId', 'match.homeTeam.name', 'match.awayTeam.name'])
    #print(features.shape)


    # Get the result and merge to the feature_df

    match_results = (afl_df.assign(result=lambda df: df.apply(lambda row: 1 if row['f_goal_diff'] > 0 else 0, axis=1)))
    # Merge result column to feature_df
    feature_df = pd.merge(features, match_results[['match.matchId', 'result']], on='match.matchId')

    return feature_df,features_rolling_averages, afl_data, features
    

In [318]:
feature_df, features_rolling_averages, afl_data, features = create_training_and_test_data(afl_df,df_next_games_teams)
feature_columns = [col for col in feature_df if col.startswith('f_')]


In [319]:

# Build model from feature_df

feature_df = feature_df.dropna()

all_X = feature_df.loc[:, feature_columns]
all_y = feature_df.loc[:, 'result']

X_train, X_test, y_train, y_test = train_test_split(all_X, all_y, test_size=0.30, random_state=42)

# Scale features
scaler = StandardScaler()
X_train[feature_columns] = scaler.fit_transform(X_train[feature_columns])
X_test[feature_columns] = scaler.transform(X_test[feature_columns])


In [320]:
# Create a list of standard classifiers
classifiers = [
    #Ensemble Methods
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),

    #Gaussian Processes
    gaussian_process.GaussianProcessClassifier(),
    
    #GLM
    linear_model.LogisticRegressionCV(),
    
    #Navies Bayes
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),
    
    #SVM
    svm.SVC(probability=True),
    svm.NuSVC(probability=True),
    
    #Discriminant Analysis
    discriminant_analysis.LinearDiscriminantAnalysis(),
    discriminant_analysis.QuadraticDiscriminantAnalysis(),

    
    #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
    XGBClassifier()    
]

# Define a functiom which finds the best algorithms for our modelling task
def find_best_algorithms(classifier_list, X, y):
    # This function is adapted from https://www.kaggle.com/yassineghouzam/titanic-top-4-with-ensemble-modeling
    # Cross validate model with Kfold stratified cross validation
    kfold = StratifiedKFold(n_splits=5)
    
    # Grab the cross validation scores for each algorithm
    cv_results = [cross_val_score(classifier, X, y, scoring = "neg_log_loss", cv = kfold) for classifier in classifier_list]
    cv_means = [cv_result.mean() * -1 for cv_result in cv_results]
    cv_std = [cv_result.std() for cv_result in cv_results]
    algorithm_names = [alg.__class__.__name__ for alg in classifiers]
    
    # Create a DataFrame of all the CV results
    cv_results = pd.DataFrame({
        "Mean Log Loss": cv_means,
        "Log Loss Std": cv_std,
        "Algorithm": algorithm_names
    })
    
    
    return cv_results.sort_values(by='Mean Log Loss').reset_index(drop=True)

In [321]:
best_algos = find_best_algorithms(classifiers, X_train, y_train)
best_algos


[09:25:26] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[09:25:26] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[09:25:26] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[09:25:27] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

,Mean Log Loss,Log Loss Std,Algorithm
0,0.673375,0.004213,NuSVC
1,0.674561,0.010223,SVC
2,0.680029,0.013741,RandomForestClassifier
3,0.682376,0.007437,LogisticRegressionCV
4,0.689850,0.002923,AdaBoostClassifier
5,0.692066,0.000458,GaussianProcessClassifier
6,0.700618,0.033103,ExtraTreesClassifier
7,0.723719,0.028737,GradientBoostingClassifier
8,0.754308,0.026693,LinearDiscriminantAnalysis
9,0.910213,0.170140,BaggingClassifier


In [322]:
kfold = StratifiedKFold(n_splits=5)
cv_scores = cross_val_score(ensemble.RandomForestClassifier(), X_train, y_train, scoring='accuracy', cv=kfold)
cv_scores.mean()

0.5690140845070422

0

In [323]:
# Try a logistic regression model and see how it performs in terms of accuracy
kfold = StratifiedKFold(n_splits=5)
cv_scores = cross_val_score(linear_model.LogisticRegressionCV(), X_train, y_train, scoring='accuracy', cv=kfold)
cv_scores.mean()

0.5690140845070422

In [347]:
#new_scorer(X_train, y_train)

kfold = StratifiedKFold(n_splits=5)
cv_scores = cross_val_score(linear_model.LogisticRegressionCV(), X_train, y_train, scoring=new_scorer, cv=kfold)
cv_scores.mean()

nan

In [344]:
kfold = StratifiedKFold(n_splits=5)
cv_scores = cross_val_score(XGBClassifier(), X_train, y_train, scoring=new_scorer, cv=kfold)
cv_scores.mean()

[09:35:30] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[09:35:30] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[09:35:30] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[09:35:31] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

nan

In [325]:
# Define a function which optimises the hyperparameters of our chosen algorithms
def optimise_hyperparameters(train_x, train_y, algorithms, parameters):
    kfold = StratifiedKFold(n_splits=5)
    best_estimators = []
    
    for alg, params in zip(algorithms, parameters):
        gs = GridSearchCV(alg, param_grid=params, cv=kfold, scoring='neg_log_loss', verbose=1)
        gs.fit(train_x, train_y)
        best_estimators.append(gs.best_estimator_)
    return best_estimators

# Define our parameters to run a grid search over
lr_grid = {
    "C": [0.0001, 0.001, 0.01, 0.05, 0.2, 0.5],
    "solver": ["newton-cg", "lbfgs", "liblinear"]
}

# Add our algorithms and parameters to lists to be used in our function
alg_list = [LogisticRegression(), XGBClassifier(), ensemble.RandomForestClassifier()]
param_list = [lr_grid]

In [326]:
# Find the best estimators, then add our other estimators which don't need optimisation
best_estimators = optimise_hyperparameters(X_train, y_train, alg_list, param_list)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


In [327]:
lr_best_params = best_estimators[0].get_params()
lr_best_params

{'C': 0.001,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'liblinear',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [328]:
kfold = StratifiedKFold(n_splits=10)
cv_scores = cross_val_score(linear_model.LogisticRegression(**lr_best_params), X_train, y_train, scoring='neg_log_loss', cv=kfold)
cv_scores.mean()

-0.6745408805217019

In [329]:
lr = LogisticRegression(**lr_best_params)
lr.fit(X_train, y_train)
final_predictions_lr = lr.predict(X_test)

accuracy = (final_predictions_lr == y_test).mean() * 100

print("Our accuracy in predicting test data is: {:.2f}%".format(accuracy))

Our accuracy in predicting test data is: 59.67%


In [330]:
xgb = XGBClassifier(**lr_best_params)
xgb.fit(X_train, y_train)
final_predictions_xgb = xgb.predict(X_test)

accuracy = (final_predictions_xgb == y_test).mean() * 100

print("Our accuracy in predicting test data is: {:.2f}%".format(accuracy))

[09:25:33] WARNING: ..\src\learner.cc:576: 
Parameters: { "C", "dual", "fit_intercept", "intercept_scaling", "max_iter", "multi_class", "penalty", "solver", "tol", "verbose", "warm_start" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[09:25:33] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Our accuracy in predicting test data is: 56.72%


In [331]:
svm_model = svm.SVC()
svm_model.fit(X_train, y_train)
final_predictions_svm = svm_model.predict(X_test)

accuracy = (final_predictions_svm == y_test).mean() * 100

print("Our accuracy in predicting test data is: {:.2f}%".format(accuracy))



Our accuracy in predicting test data is: 60.00%


In [340]:
def new_scorer(y,y_pred):
    xx = pd.DataFrame({'y':list(y), 'y_pred': list(y_pred)}, columns=['y','y_pred'])     
    xx.loc[xx['y'].astype(int)==0,'actual_score'] = 1+np.log2(1-xx['y_pred'].astype(float))
   
    xx.loc[xx['y'].astype(int)==1,'actual_score'] = 1+np.log2(xx['y_pred'].astype(float))
   
    return np.mean(xx['actual_score'])

In [333]:
#new_scorer(y_test,final_predictions)

In [356]:
importance = np.round(xgb.feature_importances_,4)
dictionary = dict(zip(feature_columns, importance))
sorted_dictionary=sorted(dictionary.items(), key=lambda x:x[1], reverse=True)
names=[]
values=[]
for i in range(0, len(importance)):
    print('Feature Importance: {:35} {}%'.format(
        sorted_dictionary[i][0], np.round(sorted_dictionary[i][1]*100,4))
         )
    names.append(sorted_dictionary[i][0])
    values.append(np.round(sorted_dictionary[i][1]*100,4))

Feature Importance: f_Goal.Assists                      4.2%
Feature Importance: f_One.Percenters                    2.88%
Feature Importance: f_Clearances                        2.56%
Feature Importance: f_homeTeamScore.minutesInFront      2.47%
Feature Importance: f_H_result_2                        2.42%
Feature Importance: f_result                            2.39%
Feature Importance: f_Contested.Possessions             2.32%
Feature Importance: f_homeTeamScore.matchScore.totalScore 2.24%
Feature Importance: f_Tackles                           2.17%
Feature Importance: f_HQ3G                              2.01%
Feature Importance: f_AQ4B                              1.99%
Feature Importance: f_homeTeamScoreChart.leftBehinds    1.96%
Feature Importance: f_H_HTGDIFF_2                       1.93%
Feature Importance: f_awayTeamScoreChart.leftPosters    1.9%
Feature Importance: f_A_AVGATGDIFF_1                    1.88%
Feature Importance: f_A_ATGDIFF_2                       1.81%
Feature 

In [334]:
next_round_features = features[features['train_data']==0][feature_columns]

next_round_predictions = lr.predict(next_round_features)


In [335]:
next_round_predictions

array([1, 1, 1, 1, 0, 0, 1, 0, 0], dtype=int64)

In [336]:
prediction_probs = lr.predict_proba(next_round_features)
prediction_probs

array([[0.2287226 , 0.7712774 ],
       [0.0016452 , 0.9983548 ],
       [0.07310986, 0.92689014],
       [0.00208952, 0.99791048],
       [0.89444607, 0.10555393],
       [0.61807734, 0.38192266],
       [0.49454688, 0.50545312],
       [0.99592871, 0.00407129],
       [0.99036463, 0.00963537]])

In [337]:

next_round_predictions_xgb = xgb.predict(next_round_features)
next_round_predictions_xgb

array([0, 1, 1, 1, 1, 1, 1, 1, 0], dtype=int64)

In [338]:

next_round_predictions_svm = svm_model.predict(next_round_features)
next_round_predictions_svm

array([1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [ ]:


# Can we train and retrain model to get max score for a Round?

# predict rounds and plot score based on model


